In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm
import re 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service


In [2]:
# 서울시 숙소 리스트 로드
df = pd.read_csv('../DAIN/crw_list.txt')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  610 non-null    int64  
 1   인허가일자       610 non-null    object 
 2   인허가취소일자     1 non-null      float64
 3   영업상태코드      610 non-null    int64  
 4   폐업일자        136 non-null    object 
 5   휴업시작일자      9 non-null      object 
 6   휴업종료일자      9 non-null      object 
 7   재개업일자       0 non-null      float64
 8   지번주소        610 non-null    object 
 9   도로명주소       602 non-null    object 
 10  도로명우편번호     516 non-null    float64
 11  사업장명        610 non-null    object 
 12  좌표정보(X)     603 non-null    float64
 13  좌표정보(Y)     603 non-null    float64
 14  위도          583 non-null    float64
 15  경도          583 non-null    float64
dtypes: float64(7), int64(2), object(7)
memory usage: 76.4+ KB


In [19]:
# 크롬 드라이버 시작
chrome_options = webdriver.ChromeOptions()              # 옵션값 사용
chrome_options.add_experimental_option("detach",True)   # 브라우저 꺼짐 방지
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install(), options = chrome_options))
#driver = webdriver.Chrome()

In [20]:
driver.get("https://map.naver.com/v5")
driver.implicitly_wait(5)
#driver.maximize_window()

In [21]:
# 검색 트리
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('이비스 앰배서더 명동', ',  ', '서울특별시 중구 명동1가 59-5')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('호텔 데님(Hotel Denim)',', ', '서울특별시 강남구 논현로 66 (개포동)')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('티마크그랜드호텔명동', ', ', '서울특별시 중구 회현동1가 194-15')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('위너스', ', ', '서울특별시 금천구 독산동 335-21')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('데이즈 호텔 명동', ', ', '서울특별시 중구 충무로1가 24-30')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('삼화', ', ', '서울특별시 관악구 신림동 1432-62')
driver.find_element(By.CLASS_NAME, 'input_search').send_keys('나이스타임 관광호텔', ', ', '서울특별시 관악구 신림동 1422-18')
driver.find_element(By.CLASS_NAME, 'input_search').send_keys(Keys.ENTER)
driver.implicitly_wait(5)
driver.switch_to.frame("searchIframe")

In [22]:
#식당 정보가 있다면 첫번째 식당의 url을 가져오기    
search_result = driver.find_elements(by=By.XPATH, value='//*[@id="_pcmap_list_scroll_container"]/ul/li')
count_search_result = len(driver.find_elements(by=By.XPATH, value='//*[@id="_pcmap_list_scroll_container"]/ul/li'))
print(f"주소검색결과 개수: {count_search_result}")
if count_search_result == 0:   
    pass
elif count_search_result > 1:
    #식당 정보 클릭        
    driver.execute_script('return document.querySelector("#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div.ouxiq.icT4K > a:nth-child(1)").click()')
    driver.implicitly_wait(2)
else:
    print('검색 결과 한개, 새창 없음')

주소검색결과 개수: 1
검색 결과 한개, 새창 없음


In [23]:
# 검색후 오른쪽 확장 탭으로 전환
driver.switch_to.default_content()
detail_iframe = driver.find_element(By.ID, 'entryIframe')
driver.switch_to.frame(detail_iframe)
# driver.switch_to.frame('entryIframe') # 그냥하면 안됨


# 별점
try:
    star_score = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
except:
    star_score = None
print("star_score: ", star_score)


# 블로그 리뷰 수 & 방문자 리뷰 수
# 방문자
try:
    visitor_review_count = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em').text
except:
    visitor_review_count = None
print("visitor_review_count: ", visitor_review_count)

# 블로그
try:
    blog_review_count = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[3]/a/em').text
except:
    blog_review_count = None
print("blog_review_count: ", blog_review_count)


# 가까운 지하철 역에서 거리 / 도보시간
# 거리
try:
    dist_from_stn = driver.find_element(By.CSS_SELECTOR, 'div.O8qbU.tQY7D > div > div > em').text
except:
    dist_from_stn = None
print("dist_from_stn: ", dist_from_stn)

# 시간
try:
    time_from_stn = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[2]/div/div/div/span').text
    print("time_from_stn: ", time_from_stn)
    if time_from_stn.find("분"):
        time_from_stn = time_from_stn.split('분')[0].split('도보')[1]
except:
    time_from_stn = None
print("time_from_stn: ", time_from_stn)


# 주요시설
facilitys_list = []
try:
    facilitys = driver.find_elements(By.CSS_SELECTOR, 'div.place_section_content > div > div.fusPl > ul > li')
    for li in facilitys:
        facilitys_list.append(li.text.strip())
except:
    facilitys_list = None
print("facilitys_list: ", facilitys_list)



star_score:  None
visitor_review_count:  None
blog_review_count:  None
dist_from_stn:  207m
time_from_stn:  None
facilitys_list:  []


In [24]:
# .. 리뷰 클릭
try:
    check_list = driver.find_elements(By.CSS_SELECTOR, 'div.place_fixed_maintab > div > div > div > div.flicking-camera > a > span.veBoZ')
except:
    pass
else:
    for i, li in enumerate(check_list):
        #print(i, type(li.text), li.text)
        if "리뷰" in li.text: 
            find_i = i+1  # python index + 1
    driver.find_element(By.CSS_SELECTOR, f'div.place_fixed_maintab > div > div > div > div.flicking-camera > a:nth-child({find_i})').click()


try:
    these_good_count = int(driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.no_margin.mdJ86 > div > div > div._Wmab > em').text)
    print('these_good_count: ', these_good_count)
    
except:
    print("이런점이 좋아요 정보가 없습니다")
    these_good_list = None
    these_good_count = None

else:
    # "이런점이 좋았어요" 더보기 다 눌러 놓기
    while True:
        try:
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > a.Tvx37').click()
            driver.implicitly_wait(1)
        except:
            break

    # 이런점이 좋아요, 딕셔너리로 {'리뷰':좋아요개수}
    these_good_list = []
    these_good = driver.find_elements(By.CSS_SELECTOR, 'div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li')
    for i, li in enumerate(these_good):
        these_good_list.append({li.text.split("\n")[0].strip().replace('"',""): int(li.text.split("\n")[2].strip())})
    print("these_good_list: ", these_good_list)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.place_fixed_maintab > div > div > div > div.flicking-camera > a:nth-child(2)"}
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F4DCE3+50899]
	(No symbol) [0x00EDE111]
	(No symbol) [0x00DE5588]
	(No symbol) [0x00E108F9]
	(No symbol) [0x00E10AFB]
	(No symbol) [0x00E3F902]
	(No symbol) [0x00E2B944]
	(No symbol) [0x00E3E01C]
	(No symbol) [0x00E2B6F6]
	(No symbol) [0x00E07708]
	(No symbol) [0x00E0886D]
	GetHandleVerifier [0x011B3EAE+2566302]
	GetHandleVerifier [0x011E92B1+2784417]
	GetHandleVerifier [0x011E327C+2759788]
	GetHandleVerifier [0x00FE5740+672048]
	(No symbol) [0x00EE8872]
	(No symbol) [0x00EE41C8]
	(No symbol) [0x00EE42AB]
	(No symbol) [0x00ED71B7]
	BaseThreadInitThunk [0x75630099+25]
	RtlGetAppContainerNamedObjectPath [0x77557B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77557B3E+238]


In [58]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="1d401a8854411add424658c3c27d516c")>>